# Last one for experiments

In [1]:
import sys
import pickle
import pandas as pd
import numpy as np
import math
from datetime import datetime
import dateutil
import copy
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression, LinearRegression

In [2]:
from coronavirus_mod_ita import *

In [3]:
sns.set(style="ticks")

# Covid19 micro founded evolutionary model

## Utils Functions, Classes & Configuration

In [4]:
__MOD_NAME = "ModNat_v10-err1-abs"
__SAVED_MOD_PATH = "saved_models/modreg_updates/"
__PRED_DAYS = 50
__DATE_CURR = '2020-03-26'

np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

## Error Formula
#Error ver 1
err_tot = (err_Igc_cum + err_Igc + err_Gc_cum + err_M_cum + err_Igci_t)/5
res = np.asarray(list(map(lambda x1, x2: (x1 - x2), array1, array2)))

# Regional & National Models Calibration

# Daily procedure

In [5]:
def get_grid_param(params, delta_mult = 3, steps = 3):
    grid_param = GridParam()
    
    for param in params.keys():
        if param in ['t1', 'tgi2', 'tgn2', 'ta2', 'Igs_t0', 'Ias_t0']:
            param_avg = params[param]
            param_min = param_avg - ((delta_mult - 1)/delta_mult)*(param_avg - 1)
            param_max = param_avg * delta_mult
            grid_param.setGrid(Param(param, par_min = 1), grid_avg = param_avg, grid_min = param_min, grid_max = param_max, steps = steps)
        elif param in ['rg_period', 'ra_period']:
            grid_param.setGridList(Param(param), [[None]])
        else: 
            param_avg = params[param]
            param_min = param_avg - ((delta_mult - 1)/delta_mult)*(param_avg)
            param_max = param_avg * delta_mult
            grid_param.setGrid(Param(param), grid_avg = param_avg, grid_min = param_min, grid_max = param_max, steps = steps)
    
    return grid_param

def load_grid(grid_name):
    grid_param = GridParam()
    if grid_name == 'default':
        grid_param.setGrid(Param('rg', par_min = 0.000001), grid_avg = 0.3, grid_min = 0.2, grid_max = 0.5, steps = 4)
        grid_param.setGrid(Param('ra', par_min = 0.000001), grid_avg = 0.6, grid_min = 0.5, grid_max = 0.9, steps = 4)
        grid_param.setGrid(Param('alpha', par_min = 0.00000001), grid_avg = 0.85, grid_min = 0.93, grid_max = 0.98, steps = 4)
        grid_param.setGrid(Param('beta', par_min = 0.00000001), grid_avg = 0.02, grid_min = 0.01, grid_max = 0.05, steps = 4)
        grid_param.setGrid(Param('beta_gcn', par_min = 0.00000001), grid_avg = 0.01, grid_min = 0.005, grid_max = 0.02, steps = 3)
        grid_param.setGrid(Param('gamma', par_min = 0.00000001), grid_avg = 0.65, grid_min = 0.45, grid_max = 0.85, steps = 3)
        grid_param.setGrid(Param('t1', par_min = 1), grid_avg = 4, grid_min = 2, grid_max = 6, steps = 3)
        grid_param.setGrid(Param('tgi2', par_min = 1), grid_avg = 8, grid_min = 6, grid_max = 15, steps = 3)
        grid_param.setGrid(Param('tgn2', par_min = 1), grid_avg = 8, grid_min = 6, grid_max = 15, steps = 3)
        grid_param.setGrid(Param('ta2', par_min = 1), grid_avg = 12, grid_min = 7, grid_max = 20, steps = 3)
        grid_param.setGrid(Param('Igs_t0', par_min = 0), grid_avg = 20, grid_min = 100, grid_max = 200, steps = 3)
        grid_param.setGrid(Param('Igci_t0', par_min = 0), grid_avg = 0, grid_min = 0, grid_max = 200, steps = 1)
        grid_param.setGrid(Param('Igcn_t0', par_min = 0), grid_avg = 0, grid_min = 0, grid_max = 200, steps = 1)
        grid_param.setGrid(Param('Ias_t0', par_min = 0), grid_avg = 20, grid_min = 100, grid_max = 200, steps = 3)
        grid_param.setGrid(Param('M_t0', par_min = 0), grid_avg = 0, grid_min = 70, grid_max = 200, steps = 1)
        grid_param.setGrid(Param('Ggci_t0', par_min = 0), grid_avg = 0, grid_min = 0, grid_max = 200, steps = 1)
        grid_param.setGrid(Param('Ggcn_t0', par_min = 0), grid_avg = 0, grid_min = 0, grid_max = 200, steps = 1)
        grid_param.setGrid(Param('Gas_t0', par_min = 0), grid_avg = 0, grid_min = 100, grid_max = 500, steps = 1)
    elif grid_name == 'default_fast':
        grid_param.setGrid(Param('rg', par_min = 0.000001), grid_avg = 0.3, grid_min = 0.2, grid_max = 0.5, steps = 2)
        grid_param.setGrid(Param('ra', par_min = 0.000001), grid_avg = 0.6, grid_min = 0.5, grid_max = 0.9, steps = 2)
        grid_param.setGrid(Param('alpha', par_min = 0.00000001), grid_avg = 0.85, grid_min = 0.93, grid_max = 0.98, steps = 2)
        grid_param.setGrid(Param('beta', par_min = 0.00000001), grid_avg = 0.02, grid_min = 0.01, grid_max = 0.05, steps = 2)
        grid_param.setGrid(Param('beta_gcn', par_min = 0.00000001), grid_avg = 0.01, grid_min = 0.005, grid_max = 0.02, steps = 2)
        grid_param.setGrid(Param('gamma', par_min = 0.00000001), grid_avg = 0.65, grid_min = 0.25, grid_max = 0.85, steps = 2)
        grid_param.setGrid(Param('t1', par_min = 1), grid_avg = 4, grid_min = 2, grid_max = 6, steps = 2)
        grid_param.setGrid(Param('tgi2', par_min = 1), grid_avg = 8, grid_min = 6, grid_max = 15, steps = 2)
        grid_param.setGrid(Param('tgn2', par_min = 1), grid_avg = 8, grid_min = 6, grid_max = 15, steps = 2)
        grid_param.setGrid(Param('ta2', par_min = 1), grid_avg = 12, grid_min = 7, grid_max = 20, steps = 2)
        grid_param.setGrid(Param('Igs_t0', par_min = 0), grid_avg = 20, grid_min = 100, grid_max = 200, steps = 1)
        grid_param.setGrid(Param('Igci_t0', par_min = 0), grid_avg = 0, grid_min = 0, grid_max = 200, steps = 1)
        grid_param.setGrid(Param('Igcn_t0', par_min = 0), grid_avg = 0, grid_min = 0, grid_max = 200, steps = 1)
        grid_param.setGrid(Param('Ias_t0', par_min = 0), grid_avg = 20, grid_min = 100, grid_max = 200, steps = 1)
        grid_param.setGrid(Param('M_t0', par_min = 0), grid_avg = 0, grid_min = 70, grid_max = 200, steps = 1)
        grid_param.setGrid(Param('Ggci_t0', par_min = 0), grid_avg = 0, grid_min = 0, grid_max = 200, steps = 1)
        grid_param.setGrid(Param('Ggcn_t0', par_min = 0), grid_avg = 0, grid_min = 0, grid_max = 200, steps = 1)
        grid_param.setGrid(Param('Gas_t0', par_min = 0), grid_avg = 0, grid_min = 100, grid_max = 500, steps = 1)
    elif grid_name == 'broad':
        grid_param.setGrid(Param('rg', par_min = 0.0001), grid_avg = 0.22, grid_min = 0.08, grid_max = 0.8, steps = 3)
        grid_param.setGrid(Param('ra', par_min = 0.0001), grid_avg = 0.55, grid_min = 0.09, grid_max = 0.8, steps = 3)
        grid_param.setGrid(Param('alpha', par_min = 0.0001, par_max = 0.999), grid_avg = 0.68, grid_min = 0.18, grid_max = 0.95, steps = 3)
        grid_param.setGrid(Param('beta', par_min = 0.00000001, par_max = 0.999), grid_avg = 0.0072, grid_min = 0.0009, grid_max = 0.11, steps = 3)
        grid_param.setGrid(Param('beta_gcn', par_min = 0.00000001, par_max = 0.999), grid_avg = 0.0065, grid_min = 0.0009, grid_max = 0.11, steps = 3)
        grid_param.setGrid(Param('gamma', par_min = 0.00000001, par_max = 0.999), grid_avg = 0.067, grid_min = 0.015, grid_max = 0.95, steps = 3)
        grid_param.setGrid(Param('t1', par_min = 2, par_max = 40), grid_avg = 10, grid_min = 2, grid_max = 30, steps = 3)
        grid_param.setGrid(Param('tgi2', par_min = 3, par_max = 40), grid_avg = 15, grid_min = 3, grid_max = 35, steps = 3)
        grid_param.setGrid(Param('tgn2', par_min = 3, par_max = 40), grid_avg = 15, grid_min = 3, grid_max = 35, steps = 3)
        grid_param.setGrid(Param('ta2', par_min = 2, par_max = 20), grid_avg = 15, grid_min = 5, grid_max = 20, steps = 3)
        grid_param.setGrid(Param('Igs_t0', par_min = 0), grid_avg = 100, grid_min = 2, grid_max = 200, steps = 2)
        grid_param.setGrid(Param('Igci_t0', par_min = 0), grid_avg = 0, grid_min = 0, grid_max = 100, steps = 1)
        grid_param.setGrid(Param('Igcn_t0', par_min = 0), grid_avg = 0, grid_min = 0, grid_max = 200, steps = 1)
        grid_param.setGrid(Param('Ias_t0', par_min = 0), grid_avg = 200, grid_min = 2, grid_max = 200, steps = 2)
        grid_param.setGrid(Param('M_t0', par_min = 0), grid_avg = 0, grid_min = 0, grid_max = 200, steps = 1)
        grid_param.setGrid(Param('Ggci_t0', par_min = 0), grid_avg = 0, grid_min = 0, grid_max = 200, steps = 1)
        grid_param.setGrid(Param('Ggcn_t0', par_min = 0), grid_avg = 0, grid_min = 0, grid_max = 200, steps = 1)
        grid_param.setGrid(Param('Gas_t0', par_min = 0), grid_avg = 0, grid_min = 0, grid_max = 300, steps = 1)
    elif grid_name == 'broad_fast':
        grid_param.setGrid(Param('rg', par_min = 0.0001), grid_avg = 0.22, grid_min = 0.08, grid_max = 0.8, steps = 2)
        grid_param.setGrid(Param('ra', par_min = 0.0001), grid_avg = 0.55, grid_min = 0.09, grid_max = 0.8, steps = 2)
        grid_param.setGrid(Param('alpha', par_min = 0.0001, par_max = 0.999), grid_avg = 0.68, grid_min = 0.18, grid_max = 0.95, steps = 2)
        grid_param.setGrid(Param('beta', par_min = 0.00000001, par_max = 0.999), grid_avg = 0.0072, grid_min = 0.0009, grid_max = 0.11, steps = 2)
        grid_param.setGrid(Param('beta_gcn', par_min = 0.00000001, par_max = 0.999), grid_avg = 0.0065, grid_min = 0.0009, grid_max = 0.11, steps = 1)
        grid_param.setGrid(Param('gamma', par_min = 0.00000001, par_max = 0.999), grid_avg = 0.067, grid_min = 0.015, grid_max = 0.95, steps = 2)
        grid_param.setGrid(Param('t1', par_min = 2, par_max = 40), grid_avg = 10, grid_min = 2, grid_max = 30, steps = 2)
        grid_param.setGrid(Param('tgi2', par_min = 3, par_max = 40), grid_avg = 15, grid_min = 3, grid_max = 35, steps = 2)
        grid_param.setGrid(Param('tgn2', par_min = 3, par_max = 40), grid_avg = 15, grid_min = 3, grid_max = 35, steps = 2)
        grid_param.setGrid(Param('ta2', par_min = 2, par_max = 20), grid_avg = 15, grid_min = 5, grid_max = 20, steps = 2)
        grid_param.setGrid(Param('Igs_t0', par_min = 0), grid_avg = 100, grid_min = 2, grid_max = 200, steps = 2)
        grid_param.setGrid(Param('Igci_t0', par_min = 0), grid_avg = 0, grid_min = 0, grid_max = 100, steps = 1)
        grid_param.setGrid(Param('Igcn_t0', par_min = 0), grid_avg = 0, grid_min = 0, grid_max = 200, steps = 1)
        grid_param.setGrid(Param('Ias_t0', par_min = 0), grid_avg = 200, grid_min = 2, grid_max = 200, steps = 2)
        grid_param.setGrid(Param('M_t0', par_min = 0), grid_avg = 0, grid_min = 0, grid_max = 200, steps = 1)
        grid_param.setGrid(Param('Ggci_t0', par_min = 0), grid_avg = 0, grid_min = 0, grid_max = 200, steps = 1)
        grid_param.setGrid(Param('Ggcn_t0', par_min = 0), grid_avg = 0, grid_min = 0, grid_max = 200, steps = 1)
        grid_param.setGrid(Param('Gas_t0', par_min = 0), grid_avg = 0, grid_min = 0, grid_max = 300, steps = 1)
        
        
    return grid_param

In [ ]:
import os

fName = "saved_models/optmod_updates/reg_mod-vx.optmod33"
opt_mod_obj = {'model': {}}
if os.path.exists(fName):
    
    with open(fName, 'rb') as f:
        
        try:
            
            opt_mod_obj = pickle.load(f)
        except:
            print("error")
            
import_data = ImportData("dpc-covid19-ita-regioni.csv")

region_list = import_data.data_regioni['codice_regione'].unique()
region_list.sort()


for region in region_list:
    data_reg = import_data.data_regioni
    data_reg_orig = import_data.data_regioni_orig
    reg_name = data_reg_orig[data_reg_orig.codice_regione == region]['denominazione_regione'].unique()[0]
    reg_pop = import_data.anagr_regioni[import_data.anagr_regioni.codice_regione == region]['pop'].unique()
    
    data_uff = ActualData(
        data_reg[data_reg.codice_regione == region].reset_index()
    )
    
    grid_param = None
    if region in opt_mod_obj['model'].keys():
        opt_mod_prev = opt_mod_obj['model'][region]['opt']
        grid_param = get_grid_param(opt_mod_prev['mod'].params, delta_mult = 3)
    else:
        grid_param = load_grid('broad')
    print("Running model for region: %d, %s, %d" %(region, reg_name, reg_pop))
    print(grid_param.paramGrid)
    mod_optimizer = ModelOptimizer(data_uff, grid_param, 400, sync_data_perc = 0.3, Pop_tot=reg_pop)
    mod_optimizer.start()
    
    opt_mod_obj['model'][region] = {'name': reg_name, 'opt': mod_optimizer.opt_model, 'opt_window': mod_optimizer.opt_model_window}
    

#Calc National Model
data_uff = ActualData(import_data.data_nazionale)
grid_param = load_grid('default')
pop_tot = import_data.anagr_regioni['pop'].sum()
print("Running model for nation: %d" %(reg_pop))
mod_optimizer = ModelOptimizer(data_uff, grid_param, 400, sync_data_perc = 0.3, Pop_tot=pop_tot)
mod_optimizer.start()
opt_mod_obj['model']['national'] = {'name': 'national', 'opt': mod_optimizer.opt_model, 'opt_window': mod_optimizer.opt_model_window}



Running model for region: 1, Piemonte, 4356406
{'rg': [0.08, 0.22, 0.8], 'ra': [0.09, 0.55, 0.8], 'alpha': [0.18, 0.68, 0.95], 'beta': [0.0009, 0.0072, 0.11], 'beta_gcn': [0.0009, 0.0065, 0.11], 'gamma': [0.015, 0.067, 0.95], 't1': [2.0, 10.0, 30], 'tgi2': [3.0, 15.0, 35], 'tgn2': [3.0, 15.0, 35], 'ta2': [5.0, 15.0, 20], 'Igs_t0': [2, 200], 'Igci_t0': [0], 'Igcn_t0': [0], 'Ias_t0': [2, 200], 'M_t0': [0], 'Ggci_t0': [0], 'Ggcn_t0': [0], 'Gas_t0': [0]}
**** Model optimization started at: 2020-03-29 14:36:26.826533
iter: 50000 in 0:11:55.562906 at 2020-03-29 14:48:22.389486
	 m:(0.220, 0.090, 0.950, 0.001, 0.001, 0.015, 2, 35, 35, 20) 	 r:(3777028.098, 6999393.779, 6166958.965, 1889968.855, 1915517.235)
iter: 100000 in 0:24:47.306609 at 2020-03-29 15:01:14.133193
	 m:(0.800, 0.550, 0.680, 0.001, 0.001, 0.067, 2, 15, 3, 15) 	 r:(20922442223.556, 44079427878.175, 25135317290.420, 14325644820.540, 10461242048.338)
**** Model optimization ended at: 2020-03-29 15:06:49.932764
	 in: 0:30:23.106

	 (4) Found FineTuner better model: M_cum (delta: 0.010000, prev: 41643.083569 -  new:40213.533489 - -0.03)
		 in 1:01:16.315534
	 (5) Found FineTuner better model: M_cum (delta: 0.010000, prev: 40213.533489 -  new:40005.069330 - -0.01)
		 in 1:02:12.252909
	 (6) Found FineTuner better model: M_cum (delta: 0.010000, prev: 40005.069330 -  new:38565.805534 - -0.04)
		 in 1:03:05.686943
	 (7) Found FineTuner better model: M_cum (delta: 0.010000, prev: 38565.805534 -  new:37017.242358 - -0.04)
		 in 1:04:00.702325
	 (8) Found FineTuner better model: M_cum (delta: 0.010000, prev: 37017.242358 -  new:36535.061985 - -0.01)
		 in 1:04:56.319544
	 (9) Found FineTuner better model: M_cum (delta: 0.010000, prev: 36535.061985 -  new:34668.733264 - -0.05)
		 in 1:05:50.635516
	 (10) Found FineTuner better model: M_cum (delta: 0.010000, prev: 34668.733264 -  new:34210.232441 - -0.01)
		 in 1:06:47.907535
	 (11) Found FineTuner better model: M_cum (delta: 0.010000, prev: 34210.232441 -  new:32145.971

	 Windows last saving (1, 21, tot, error_prev=29358.918749, error_new=29358.918749)
	 It is not optimal, save the old one
	 Saved TunerWindow model: tot (windows: 3, last window: 21, prev: 29358.918749 new:29358.918749)
	 (7, 0) Found TunerWindow model: Igc_cum (delta: 0.010000, prev: 56889.390099 new:56537.849100 -0.01)
	 (21, 0) Found TunerWindow model: Igc_cum (delta: 0.010000, prev: 56537.849100 new:53972.691592 -0.05)
	 (21, 1) Found TunerWindow model: Igc_cum (delta: 0.010000, prev: 53972.691592 new:52601.764740 -0.03)
	 (21, 2) Found TunerWindow model: Igc_cum (delta: 0.010000, prev: 52601.764740 new:52296.648947 -0.01)
	 Windows last saving (1, 21, Igc_cum, error_prev=52296.648947, error_new=52296.648947)
	 It is not optimal, save the old one
	 Saved TunerWindow model: Igc_cum (windows: 3, last window: 21, prev: 52296.648947 new:52296.648947)
	 (0, 0) Found TunerWindow model: Igc (delta: 0.010000, prev: 57640.286316 new:55726.116499 -0.03)
	 (7, 0) Found TunerWindow model: Igc 

	 (1) Found FineTuner better model: Igc_cum (delta: 0.050000, prev: 1036.552246 -  new:1007.285697 - -0.03)
		 in 0:12:24.392391
	 (2) Found FineTuner better model: Igc_cum (delta: 0.050000, prev: 1007.285697 -  new:981.675307 - -0.03)
		 in 0:13:32.299851
	 (3) Found FineTuner better model: Igc_cum (delta: 0.050000, prev: 981.675307 -  new:978.431016 - -0.00)
		 in 0:14:39.374626
	 (4) Found FineTuner better model: Igc_cum (delta: 0.050000, prev: 978.431016 -  new:973.222301 - -0.01)
		 in 0:15:40.327149
	 (5) Found FineTuner better model: Igc_cum (delta: 0.050000, prev: 973.222301 -  new:967.705889 - -0.01)
		 in 0:16:51.244441
	 (6) Found FineTuner better model: Igc_cum (delta: 0.050000, prev: 967.705889 -  new:943.824409 - -0.02)
		 in 0:18:01.766732
	 (7) Found FineTuner better model: Igc_cum (delta: 0.050000, prev: 943.824409 -  new:924.261638 - -0.02)
		 in 0:19:19.040178
	 (1) Found FineTuner better model: Igc (delta: 0.050000, prev: 1178.056162 -  new:1129.028993 - -0.04)
		 i

In [ ]:
# Saving the models
#this function is already in library
def saveOptMod(obj, actual_data_update, path = "", mod_name="generic", bDataUpdateTime = True, bRunDatetime = True):
    
    #data_uff.date[len(data_uff.date)-1]
    actual_data_update = np.datetime_as_string(actual_data_update, unit='D')
    current_date = np.datetime_as_string(np.datetime64(datetime.now()), unit='s')
    path_full = path + mod_name
    if bDataUpdateTime: path_full = path_full + "_act-" + actual_data_update
    if bRunDatetime: path_full = path_full + "_" + current_date 
    with open(path_full, 'wb') as saved_file:
        pickle.dump(obj, saved_file)


saveOptMod(opt_mod_obj, data_uff.date.max(), path = __SAVED_MOD_PATH, mod_name = __MOD_NAME)
saveOptMod(opt_mod_obj, data_uff.date.max(), path = __SAVED_MOD_PATH, mod_name = __MOD_NAME, 
           bDataUpdateTime = False,
           bRunDatetime = False
          )

# Load saved model

fName = __SAVED_MOD_PATH + "modreg_updates/ModReg_v9-fast"
with open(fName, 'rb') as f:

    try:

        opt_mod_obj = pickle.load(f)
    except:
        print("error")

# Calibration Study
Understand calibration impacts on models

## Parameter sensibility
for all combination of parameters, plot all regions (think of scatter plot to have 2 dims = two parameter per plot). Color is the value of the error.

In [ ]:
mod_type = 'opt'
mod_name = 'tot'
geo_list = opt_mod_obj['model'].values()
data = {
# This needs to be fixed above
    'area': [x['name'][0] for x in geo_list],
    'rg': [x[mod_type][mod_name]['mod'].params['rg'] for x in geo_list],
    'ra': [x[mod_type][mod_name]['mod'].params['ra'] for x in geo_list],
    'alpha': [x[mod_type][mod_name]['mod'].params['alpha'] for x in geo_list],
    'beta': [x[mod_type][mod_name]['mod'].params['beta'] for x in geo_list],
    'beta_gcn': [x[mod_type][mod_name]['mod'].params['beta_gcn'] for x in geo_list],
    'gamma': [x[mod_type][mod_name]['mod'].params['gamma'] for x in geo_list],
    't1': [x[mod_type][mod_name]['mod'].params['t1'] for x in geo_list],
    'tgi2': [x[mod_type][mod_name]['mod'].params['tgi2'] for x in geo_list],
    'tgn2': [x[mod_type][mod_name]['mod'].params['tgn2'] for x in geo_list],
    'ta2': [x[mod_type][mod_name]['mod'].params['ta2'] for x in geo_list],
    'Igs_t': [x[mod_type][mod_name]['mod'].Igs_t[0] for x in geo_list],
    'Ias_t': [x[mod_type][mod_name]['mod'].Ias_t[0] for x in geo_list],
    'err_tot': [x[mod_type][mod_name]['err_tot'] for x in geo_list]
}
#for geo in opt_mod_obj['model']:
#    data['area'] = geo
df = pd.DataFrame(data)


mod_type = 'opt_window'
mod_name = 'tot'
geo_list = opt_mod_obj['model'].values()
data_window = {
# This needs to be fixed above
    'area': [x['name'][0] for x in geo_list],
    'rg': [x[mod_type][mod_name]['mod'].params['rg'] for x in geo_list],
    'ra': [x[mod_type][mod_name]['mod'].params['ra'] for x in geo_list],
    'alpha': [x[mod_type][mod_name]['mod'].params['alpha'] for x in geo_list],
    'beta': [x[mod_type][mod_name]['mod'].params['beta'] for x in geo_list],
    'beta_gcn': [x[mod_type][mod_name]['mod'].params['beta_gcn'] for x in geo_list],
    'gamma': [x[mod_type][mod_name]['mod'].params['gamma'] for x in geo_list],
    't1': [x[mod_type][mod_name]['mod'].params['t1'] for x in geo_list],
    'tgi2': [x[mod_type][mod_name]['mod'].params['tgi2'] for x in geo_list],
    'tgn2': [x[mod_type][mod_name]['mod'].params['tgn2'] for x in geo_list],
    'ta2': [x[mod_type][mod_name]['mod'].params['ta2'] for x in geo_list],
    'Igs_t': [x[mod_type][mod_name]['mod'].Igs_t[0] for x in geo_list],
    'Ias_t': [x[mod_type][mod_name]['mod'].Ias_t[0] for x in geo_list],
    'err_tot': [x[mod_type][mod_name]['err_tot'] for x in geo_list]
}
#for geo in opt_mod_obj['model']:
#    data['area'] = geo
df_window = pd.DataFrame(data_window)

In [ ]:
df.sort_values('rg')

In [ ]:
df_window.sort_values('rg')

In [ ]:

mod_name = 'tot'
geo = 'national'
param = 'alpha'
#df_window[param] - df[param] 

print(opt_mod_obj['model'][geo]['name'])
opt_mod_obj['model'][geo]['opt_window'][mod_name]['mod'].ra_period
#opt_mod_obj['model']

In [ ]:
(df['err_tot'].median(), df['err_tot'].mean(), df['err_tot'].std(), df['err_tot'].min(), df['err_tot'].max())

In [ ]:
def categ(x, cat1, cat2, cat3):
    res = ''
    if x < cat1: res =  'low'
    if x >= cat1 and x < cat2 : res = 'med'
    if x >= cat2 and x < cat3 : res = 'high'
    if x > cat3: res = 'very high'
    return res

df['err_tot_cat'] = [categ(x, 0.08, 0.2, 0.3) for x in df['err_tot'].to_list()]

 1. Check if a saved model exists in saved_models/optmod_updates as 'reg_mod-vx.optmod'.
    1. If exists: load in opt_mod_obj_prev
    2. if not: load default grid, 
2. for each region:
    1. set the grid by:
        1. if region exists in opt_mod_obj: create the grid centered on the optimal value of the same region from opt_mod_obj 
        2. Else: grid is set to the default grid
    2. perform optimization procedure
    3. update opt_mod_obj with the new region object
3. Same as region for national model
4. Update opt_mod_obj.national_aggr with mod_data creating by summing regional models
5. Update act_data_lastdate with max date of act_data
6. Save the opt_mod_obj in /optmod_updates as 'reg_mod-vx_lastdate_rundate' and 'reg_mod-vx'
        

# Continuous improvement model optimization

Goal: create a procedure that continuously perform optimization of a starting optimal model already available at the beginning of the procedure. The procedure also takes into account new actual data coming in, and adjust accordingly to avoid problems in the selection of optimal model by comparing to previous models trained on old data.

1. Load an initial regional model object, calculated as described above
2. WHILE: cicla infinitamente per trovare nuovi modelli, performando una perturbazione randomica (uniformemente distribuita in un dato intorno configurabile) su n parametri scelti a caso (n randomico con intervalli configurabili, suggerito tra 1 e 3)
    0. controlla se c'è stato un aggiornamento dei dati actual. Nel caso prendi i parametri del modello ottimale ultimo, e esegui la procedura di ottimizzazione e sovrascrivi il modello ottimale gelerale. Se non c'è stato il cambio, esegui le seguenti istruzioni:
        1. prendi il modello inizialmente caricato, ricava i parametri ottimi di opt_mod[tot]
        2. calcola i parametri di perturbazione randomici e calcola la nuova griglia
        3. performa ottimizzazione con la nuova griglia
        4. confronta il opt_mod e opt_window nuovi ottenuti con i corrispettivi iniziali (prima della perturbazione) e aggiorna il modello ottimale generale se ci sono miglioramenti.
    

## 2. Model Run

In [ ]:
"""
region_code = 'national'
print("Regione: ", opt_mod_obj['model'][region_code]['name'])
data_uff = ActualData(
        import_data.data_regioni[import_data.data_regioni.codice_regione == region_code].reset_index()
    )
opt_model = opt_mod_obj['model'][region_code]['opt']
opt_model_window = opt_mod_obj['model'][region_code]['opt_window']
"""
data_uff = ActualData(import_data.data_nazionale)
opt_model = opt_mod_obj['model']['national']['opt']
opt_model_window = opt_mod_obj['model']['national']['opt_window']

In [ ]:
model_name = 'tot'
params_opt = opt_model[model_name]['mod'].params
params_opt_window = opt_model_window[model_name]['mod'].params
for param_name in params_opt.keys():
    if param_name not in ['rg_period', 'ra_period']:
        print("%10s: %7.4f  %7.4f" %(param_name, params_opt[param_name], params_opt_window[param_name]))

# Model Statistics & Predictions

In [ ]:
date_curr = __DATE_CURR
date_curr = '2020-03-26'
#model_name = 'tot'
stats = ModelStats(opt_model[model_name], data_uff)
stats.printKpis(date_curr)

In [ ]:
stats = ModelStats(opt_model_window[model_name], data_uff)
stats.printKpis(date_curr)

In [ ]:
pred_days = 7
for var_name in var_dic.keys():
    for optmod_name in optmod_dic.keys():
        predNextDays(optmod_name, opt_model, var_name, pred_days)
        print()

In [ ]:
pred_days = 7
for var_name in var_dic.keys():
    for optmod_name in optmod_dic.keys():
        predNextDays(optmod_name, opt_model_window, var_name, pred_days)
        print()

# Model Performance

## Optimal Model

In [ ]:
#Models dynamics
pred_days = __PRED_DAYS
#pred_days = 10
graphOptAll('tot', opt_model, data_uff, var_name_uff = 'dat_Igc_cum', pred_days=pred_days)
graphOptAll('Igc_cum', opt_model, data_uff, var_name_uff = 'dat_Igc_cum', pred_days=pred_days)
graphOptAll('Igc', opt_model, data_uff, var_name_uff = 'dat_Igc_cum', pred_days=pred_days)
graphOptAll('Gc_cum', opt_model, data_uff, var_name_uff = 'dat_Igc_cum', pred_days=pred_days)
graphOptAll('M_cum', opt_model, data_uff, var_name_uff = 'dat_Igc_cum', pred_days=pred_days)

In [ ]:
def graphAllCompAct(var_name, opt_mod, data_uff):
    #var_name = 'dat_Igc_cum'
    #opt_mod = mod_optimizer.opt_model


    plt.figure(figsize = (10,7))
    #data_uff_istart = data_uff.i_start


    plt.plot(getattr(data_uff, var_name)[data_uff.i_start:], label = "UFF")
    plotOptModLine(var_name, opt_mod, 'tot', 'tot')
    plotOptModLine(var_name, opt_mod, 'Igc_cum', 'Igc_cum')
    plotOptModLine(var_name, opt_mod, 'Igc', 'Igc')
    #plotOptModLine(var_name, opt_mod, 'Gc_cum', 'Gc_cum')
    plotOptModLine(var_name, opt_mod, 'M_cum', 'M_cum')
    #plt.ylim((0,20000))
    plt.xlabel('giorni')
    plt.ylabel('n. persone')
    plt.title('Actual vs Model: ' + var_dic[var_name])
    plt.legend()
    plt.show()

In [ ]:
# One variable per comparison, actual vs all models
graphAllCompAct('dat_Igc_cum', opt_model, data_uff)
graphAllCompAct('dat_Igc', opt_model, data_uff)
graphAllCompAct('dat_Igci_t', opt_model, data_uff)
graphAllCompAct('dat_Igcn_t', opt_model, data_uff)
graphAllCompAct('dat_Gc_cum', opt_model, data_uff)
graphAllCompAct('dat_M_cum', opt_model, data_uff)

## Optimized Window Model

In [ ]:
#Models dynamics
#pred_days = 100
pred_days = __PRED_DAYS
#pred_days = 10
graphOptAll('tot', opt_model_window, data_uff, var_name_uff = 'dat_Igc_cum', pred_days=pred_days)
graphOptAll('Igc_cum', opt_model_window, data_uff, var_name_uff = 'dat_Igc_cum', pred_days=pred_days)
graphOptAll('Igc', opt_model_window, data_uff, var_name_uff = 'dat_Igc_cum', pred_days=pred_days)
graphOptAll('Gc_cum', opt_model_window, data_uff, var_name_uff = 'dat_Igc_cum', pred_days=pred_days)
graphOptAll('M_cum', opt_model_window, data_uff, var_name_uff = 'dat_Igc_cum', pred_days=pred_days)

In [ ]:
# One variable per comparison, actual vs all models

graphAllCompAct('dat_Igc_cum', opt_model_window, data_uff)
graphAllCompAct('dat_Igc', opt_model_window, data_uff)
graphAllCompAct('dat_Igci_t', opt_model_window, data_uff)
graphAllCompAct('dat_Igcn_t', opt_model_window, data_uff)
graphAllCompAct('dat_Gc_cum', opt_model_window, data_uff)
graphAllCompAct('dat_M_cum', opt_model_window, data_uff)

In [ ]:
mod_name = 'tot'
graphWindowComp("dat_Igc_cum", data_uff, opt_model, opt_model_window, mod_name)
graphWindowComp("dat_Igc", data_uff, opt_model, opt_model_window, mod_name)
graphWindowComp("dat_Igci_t", data_uff, opt_model, opt_model_window, mod_name)
graphWindowComp("dat_Igcn_t", data_uff, opt_model, opt_model_window, mod_name)
graphWindowComp("dat_Gc_cum", data_uff, opt_model, opt_model_window, mod_name)
graphWindowComp("dat_M_cum", data_uff, opt_model, opt_model_window, mod_name)

In [ ]:

stats.data[['uff_Igc_cum', 'mod_Igc_cum', 'uff_Igc', 'mod_Igc', 'uff_Gc_cum', 'mod_Gc_cum', 'uff_M_cum', 'mod_M_cum']]

### Model Statistics Classes

Basics Stats
1. Valore picco e corrispondente data per serie storica
2. totale morti, totale guariti, totale infettati
3. Model Graph (tutte le curve del modello)
NB: pensa a mettere i calcoli nel modello. Questa classe la usi più per mostrare le statistiche
- creare una serie storica di una data variabile, al variare di un valore di parametro --> fare grafico


Finire Windows e trovare per ogni modello la lista di rg e ra ottimale.


Appena finito qui:
- prendere dati per regioni, e addestrare un modello per regione
- fare modello nazionale come somma dei modelli regionali, comparare con efficacia modello nazionale

Inoltre, fare procedura che scarica dati da github, e poi lancia il notebook

In [ ]:
#backup

grid_param.setGrid(Param('rg', par_min = 0.000001), grid_avg = 0.25, grid_min = 0.15, grid_max = 0.35, steps = 2)
grid_param.setGrid(Param('ra', par_min = 0.000001), grid_avg = 0.4, grid_min = 0.3, grid_max = 0.5, steps = 2)
grid_param.setGrid(Param('alpha', par_min = 0.00000001), grid_avg = 0.85, grid_min = 0.75, grid_max = 0.95, steps = 2)
grid_param.setGrid(Param('beta', par_min = 0.00000001), grid_avg = 0.05, grid_min = 0.01, grid_max = 0.1, steps = 2)
grid_param.setGrid(Param('beta_gcn', par_min = 0.00000001), grid_avg = 0.05, grid_min = 0.01, grid_max = 0.1, steps = 1)
grid_param.setGrid(Param('gamma', par_min = 0.00000001), grid_avg = 0.05, grid_min = 0.01, grid_max = 0.15, steps = 2)
grid_param.setGrid(Param('t1', par_min = 1), grid_avg = 4, grid_min = 2, grid_max = 6, steps = 2)
grid_param.setGrid(Param('tgi2', par_min = 1), grid_avg = 12, grid_min = 7, grid_max = 25, steps = 2)
grid_param.setGrid(Param('tgn2', par_min = 1), grid_avg = 12, grid_min = 7, grid_max = 25, steps = 2)
grid_param.setGrid(Param('ta2', par_min = 1), grid_avg = 6, grid_min = 4, grid_max = 12, steps = 2)
grid_param.setGrid(Param('Igs_t0', par_min = 0), grid_avg = 5, grid_min = 20, grid_max = 300, steps = 1)
grid_param.setGrid(Param('Igci_t0', par_min = 0), grid_avg = 0, grid_min = 0, grid_max = 200, steps = 1)
grid_param.setGrid(Param('Igcn_t0', par_min = 0), grid_avg = 0, grid_min = 0, grid_max = 200, steps = 1)
grid_param.setGrid(Param('Ias_t0', par_min = 0), grid_avg = 5, grid_min = 1, grid_max = 20, steps = 1)
grid_param.setGrid(Param('M_t0', par_min = 0), grid_avg = 0, grid_min = 70, grid_max = 200, steps = 1)
grid_param.setGrid(Param('Ggci_t0', par_min = 0), grid_avg = 0, grid_min = 0, grid_max = 200, steps = 1)
grid_param.setGrid(Param('Ggcn_t0', par_min = 0), grid_avg = 0, grid_min = 0, grid_max = 200, steps = 1)
grid_param.setGrid(Param('Gas_t0', par_min = 0), grid_avg = 0, grid_min = 100, grid_max = 500, steps = 1)


grid_param.setGrid(Param('rg', par_min = 0.000001), grid_avg = 0.0121253, grid_min = 0.05, grid_max = 0.25, steps = 2)
grid_param.setGrid(Param('ra', par_min = 0.000001), grid_avg = 0.4332332, grid_min = 0.3, grid_max = 0.6, steps = 2)
grid_param.setGrid(Param('alpha', par_min = 0.00000001), grid_avg = 0.55079215, grid_min = 0.35, grid_max = 0.75, steps = 2)
grid_param.setGrid(Param('beta', par_min = 0.00000001), grid_avg = 0.009436764, grid_min = 0.005, grid_max = 0.1, steps = 2)
grid_param.setGrid(Param('beta_gcn', par_min = 0.00000001), grid_avg = 0.013401648500625002, grid_min = 0.005, grid_max = 0.1, steps = 2)
grid_param.setGrid(Param('gamma', par_min = 0.00000001), grid_avg = 0.061851283003125, grid_min = 0.01, grid_max = 0.15, steps = 2)
grid_param.setGrid(Param('t1', par_min = 1), grid_avg = 3, grid_min = 1, grid_max = 6, steps = 2)
grid_param.setGrid(Param('tgi2', par_min = 1), grid_avg =5, grid_min = 3, grid_max = 15, steps = 2)
grid_param.setGrid(Param('tgn2', par_min = 1), grid_avg = 12, grid_min = 7, grid_max = 25, steps = 2)
grid_param.setGrid(Param('ta2', par_min = 1), grid_avg = 15, grid_min = 7, grid_max = 25, steps = 2)
grid_param.setGrid(Param('Igs_t0', par_min = 0), grid_avg = 5, grid_min = 1, grid_max = 10, steps = 1)
grid_param.setGrid(Param('Igci_t0', par_min = 0), grid_avg = 0, grid_min = 0, grid_max = 200, steps = 1)
grid_param.setGrid(Param('Igcn_t0', par_min = 0), grid_avg = 0, grid_min = 0, grid_max = 200, steps = 1)
grid_param.setGrid(Param('Ias_t0', par_min = 0), grid_avg = 5, grid_min = 1, grid_max = 10, steps = 1)
grid_param.setGrid(Param('M_t0', par_min = 0), grid_avg = 0, grid_min = 0, grid_max = 200, steps = 1)
grid_param.setGrid(Param('Ggci_t0', par_min = 0), grid_avg = 0, grid_min = 0, grid_max = 200, steps = 1)
grid_param.setGrid(Param('Ggcn_t0', par_min = 0), grid_avg = 0, grid_min = 0, grid_max = 200, steps = 1)
grid_param.setGrid(Param('Gas_t0', par_min = 0), grid_avg = 0, grid_min = 100, grid_max = 500, steps = 1)


grid_param.setGrid(Param('rg', par_min = 0.000001), grid_avg = 0.55, grid_min = 0.01, grid_max = 1, steps = 4)
grid_param.setGrid(Param('ra', par_min = 0.000001), grid_avg = 0.55, grid_min = 0.01, grid_max = 1, steps = 4)
grid_param.setGrid(Param('alpha', par_min = 0.00000001), grid_avg = 0.55, grid_min = 0.05, grid_max = 0.95, steps = 4)
grid_param.setGrid(Param('beta', par_min = 0.00000001), grid_avg = 0.55, grid_min = 0.01, grid_max = 0.95, steps = 4)
grid_param.setGrid(Param('beta_gcn', par_min = 0.00000001), grid_avg = 0.55, grid_min = 0.01, grid_max = 0.95, steps = 4)
grid_param.setGrid(Param('gamma', par_min = 0.00000001), grid_avg = 0.55, grid_min = 0.05, grid_max = 0.95, steps = 4)
grid_param.setGrid(Param('t1', par_min = 1), grid_avg = 10, grid_min = 2, grid_max = 30, steps = 4)
grid_param.setGrid(Param('tgi2', par_min = 1), grid_avg = 15, grid_min = 2, grid_max = 35, steps = 4)
grid_param.setGrid(Param('tgn2', par_min = 1), grid_avg = 15, grid_min = 2, grid_max = 35, steps = 4)
grid_param.setGrid(Param('ta2', par_min = 1), grid_avg = 15, grid_min = 2, grid_max = 35, steps = 4)
grid_param.setGrid(Param('Igs_t0', par_min = 0), grid_avg = 100, grid_min = 5, grid_max = 20, steps = 2)
grid_param.setGrid(Param('Igci_t0', par_min = 0), grid_avg = 0, grid_min = 0, grid_max = 200, steps = 1)
grid_param.setGrid(Param('Igcn_t0', par_min = 0), grid_avg = 0, grid_min = 0, grid_max = 200, steps = 1)
grid_param.setGrid(Param('Ias_t0', par_min = 0), grid_avg = 200, grid_min = 5, grid_max = 20, steps = 2)
grid_param.setGrid(Param('M_t0', par_min = 0), grid_avg = 0, grid_min = 0, grid_max = 200, steps = 1)
grid_param.setGrid(Param('Ggci_t0', par_min = 0), grid_avg = 0, grid_min = 0, grid_max = 200, steps = 1)
grid_param.setGrid(Param('Ggcn_t0', par_min = 0), grid_avg = 0, grid_min = 0, grid_max = 200, steps = 1)
grid_param.setGrid(Param('Gas_t0', par_min = 0), grid_avg = 0, grid_min = 0, grid_max = 300, steps = 1)

